In [8]:
#import tuxml
import pandas as pd 
import warnings
warnings.filterwarnings('ignore')

#definitions
systems = ['7z','BerkeleyDBC','Dune','Hipacc','Irzip','LLVM','Polly','x264']

samples_config = ['distBased', 'divDistBased','henard', 'random', 'solverBased', 'twise']
size_methods = ["Performance"]


In [9]:
#learning a model by using linear regression
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import ElasticNet

def run_regressorML(reg, test_size, size_target, x_train, x_test, y_train, y_test):
    assert(size_target in size_methods)
    reg.fit(x_train, y_train)
    y_pred = reg.predict(x_test)
    #y_pred = reg.intercept_ + np.sum(reg.coef_ * x_test.values, axis=1)
    dfErrors = pd.DataFrame({'Actual':y_test, 'Predicted':y_pred, "error":(y_pred - y_test).abs(), "% error":((y_pred - y_test)/y_test).abs()*100})
    return dfErrors["% error"].describe()

In [10]:
def ft_importances(coef_, col):
    importanceSeries = pd.Series(coef_, index=col.values)
    return importanceSeries[importanceSeries != 0].abs().sort_values(ascending = False)

In [11]:
df_result  = pd.DataFrame(columns = ['algorithm_name', 'ft_dummyfication', 'origin_ft_selection', 'ft_selection', 'hyperparameters', 'size_target', 'test_size', 'coef_order', 'accuracy'])
df_result

,algorithm_name,ft_dummyfication,origin_ft_selection,ft_selection,hyperparameters,size_target,test_size,coef_order,accuracy


In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import ElasticNet
from sklearn import linear_model
from sklearn import metrics
from sklearn import ensemble

%run feature_importance-all_systems.ipynb



def run_Algorithms(df, ft_importance_enable, system=None): 
    
    for config_name in samples_config:
        
        path_name = config_name+"_t3"
        if(system==None):
            path = r"sampledConfigurations_"+path_name+".csv"
        else:
            path = system+"/sampledConfigurations_"+path_name+".csv"

        for samplePerc in [0.3,0.4,0.5,0.6]:

            df = pd.read_csv(path, sep=';')

            #Run fetures engines: #yes and encoding
            df = run_feature_encoding(run_features_engine_yes(df))

            path_name = config_name+"_t3_features_engine"


            #number of features after features engine execution
            num_features = round(samplePerc*len(df.columns))

            start_time_sample = pd.Timestamp.now()
            
            
            #Run feature selection with and without hyperparams (after features engine execution)
            if system is None:
#                 run_featureSelection(df, None, None, path_name)
                run_featureSelection(df, None, hyperparams_RF, path_name)
                df_importance = pd.read_csv("feature_importance_"+path_name+".csv")
            else:
#                 run_featureSelection(df, system, None, path_name)
                run_featureSelection(df, system, hyperparams_RF, path_name)
                df_importance = pd.read_csv(system+"/feature_importance_"+path_name+".csv")


            
            print("Sample features number: "+ str(num_features))
            df_importance = df[df_importance[:num_features]["Unnamed: 0"].values]
            df_importance.head()


            target = "Performance"
            paramsLasso = {'alpha': [25,10,4,2,1.0,0.8,0.5,0.3,0.2,0.1,0.05,0.02,0.01],
                           'max_iter': [1, 5, 10, 100, 1000],}
            paramsRidge = {'alpha': [25,10,4,2,1.0,0.8,0.5,0.3,0.2,0.1,0.05,0.02,0.01]}
            paramsENet = {'alpha': [25,10,4,2,1.0,0.8,0.5,0.3,0.2,0.1,0.05,0.02,0.01],
                          'l1_ratio': [0.0, 0.5, 1.0, 0.1],
                          'max_iter': [1, 5, 10, 100, 1000],}
            paramsRF =  {"max_depth":18,"n_estimators":50,"verbose":2}

#             for ft_dummyfication in [False, True]:
#                 if ft_dummyfication:
#                     #Creating dummy variables in pandas
#                     df_dummy = pd.get_dummies(df, columns=list(df.drop(columns=size_methods).columns.values))
#                     df = df.concat([df, df_dummy], axis=1)
#                     print('DF IMPORTANCE')
#                     display(df_importance)
                    
#                     df_importance_dummmy = pd.get_dummies(df_importance, columns=list(df_importance.columns.values))
                    
#             path_name = path_name + "_dummy_" 

            if ft_importance_enable:
                ft_selection =  {'None': df.drop(columns=size_methods).columns,
                                'RandomForest': df_importance.columns[:num_features],}

            else:
                ft_selection =  {'None': df.drop(columns=size_methods).columns,
                                     'RandomForest': df.columns[:num_features],}

            for origin_ft_selection in ft_selection:

                for size in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]:
                    config_start_time = pd.Timestamp.now()

                    # size = number of configurations
                    x_train, x_test, y_train, y_test = train_test_split(df[ft_selection[origin_ft_selection]], df[target], test_size=size, random_state=42)  
                    x_train, y_train = np.array(x_train.values), np.array(y_train.values)


                    if len(x_train) > 3:

                        model = {'LinearRegression': LinearRegression(),
                                 'Lasso': GridSearchCV(linear_model.Lasso(), param_grid=paramsLasso, cv=3).fit(x_train, y_train).best_estimator_,
                                 'Ridge': GridSearchCV(linear_model.Ridge(), param_grid=paramsRidge, cv=3).fit(x_train, y_train).best_estimator_,
                                 'ElasticNet': GridSearchCV(ElasticNet(), param_grid=paramsENet, scoring='r2', cv=3).fit(x_train, y_train).best_estimator_,
                                 'PolynomialRegression': LinearRegression(),
                                 'RandomForest': ensemble.RandomForestRegressor().fit(x_train, y_train)}
                    else:
                        if len(x_train) >= 2:
                            model = {'LinearRegression': LinearRegression(),
                                     'Lasso': GridSearchCV(linear_model.Lasso(), param_grid=paramsLasso, cv=2).fit(x_train, y_train).best_estimator_,
                                     'Ridge': GridSearchCV(linear_model.Ridge(), param_grid=paramsRidge, cv=2).fit(x_train, y_train).best_estimator_,
                                     'ElasticNet': GridSearchCV(ElasticNet(), param_grid=paramsENet, scoring='r2', cv=2).fit(x_train, y_train).best_estimator_,
                                     'PolynomialRegression': LinearRegression(),
                                     'RandomForest': ensemble.RandomForestRegressor().fit(x_train, y_train)}

                    for key in model:
                        
                        if (key == 'PolynomialRegression') and (origin_ft_selection == 'None'):
                            print("It doesn't scale")
                        else:
                            if (key == 'PolynomialRegression'):
                                #for 2 options we have: 𝑓(𝑥₁, 𝑥₂) = 𝑏₀ + 𝑏₁𝑥₁ + 𝑏₂𝑥₂ + 𝑏₃𝑥₁² + 𝑏₄𝑥₁𝑥₂ + 𝑏₅𝑥₂²
                                x_train = PolynomialFeatures(degree=2, include_bias=False).fit_transform(x_train)
                                x_test = PolynomialFeatures(degree=2, include_bias=False).fit_transform(x_test)

                            res_model = run_regressorML(model[key], test_size=size, size_target=target, x_train=x_train, x_test=x_test, y_train=y_train, y_test=y_test)

                            if (key == 'PolynomialRegression'):
                                model[key].coef_ = model[key].coef_[:num_features]
                                
                            if (key == 'RandomForest'):
#                                     y_pred = model[key].predict(x_test)
#                                     dfErrors = pd.DataFrame({'Actual':y_test, 'Predicted':y_pred, "error":(y_pred - y_test).abs(), "% error":((y_pred - y_test)/y_test).abs()*100})
#                                     res_model = dfErrors["% error"].describe()
#                                     res_model = model[key]
                                coef_order = ft_importances(res_model[:num_features], col=ft_selection[origin_ft_selection])
                            else:
                                coef_order = ft_importances(model[key].coef_, col=ft_selection[origin_ft_selection])

                        display(res_model)
#                             df_result.loc[len(df_result)] = [key, ft_dummyfication, origin_ft_selection, ft_selection[origin_ft_selection], model[key], target, size, coef_order, res_model]
                        df_result.loc[len(df_result)] = [key, False, origin_ft_selection, ft_selection[origin_ft_selection], model[key], target, size, coef_order, res_model]

                        config_end_time = pd.Timestamp.now()
                        exec_time = [config_name,samplePerc,num_features,ft_importance_enable,origin_ft_selection, key, size,False,config_start_time,config_end_time]
    #             exec_time = [config_name,samplePerc,num_features,ft_importance_enable,origin_ft_selection, key, size,ft_dummyfication,config_start_time,config_end_time]
                        df_time.loc[df_time.size] = exec_time
                    

            sample_end_time = pd.Timestamp.now()
            exec_time = [config_name,samplePerc,num_features,ft_importance_enable,origin_ft_selection, key, None,None,start_time_sample,sample_end_time]
            df_time.loc[df_time.size] = exec_time
            display(exec_time)

            if ft_importance_enable:
                df_result.to_csv("results/"+system+"/"+system+"-results_with_feature_importance_"+path_name+"_"+str(num_features)+".csv", header=True)
            else:
                df_result.to_csv("results/"+system+"/"+system+"-results_"+path_name+"_"+str(num_features)+".csv", header=True)

        display(df_result)
                                                                                                                                  
    if ft_importance_enable:
        time_file_name = "results/"+system+"/"+system+"-simulation_time_information-with_feature_importance.csv"
    else:
        time_file_name = "results/"+system+"/"+system+"-simulation_time_information.csv"
    
    df_time.to_csv(time_file_name)
        
    return df_result

In [13]:
df_time = pd.DataFrame(columns=['SampleAlgorithm','%FeatureSelection','#Features','FeatureImportance','OriginalFS','Algorithm','%Configurations','Dummy','StartTime', 'EndTime'])

for system in systems:
    print(system)
    
    print("\n\n-----------------------------------------------------------------------------")
    print("\n--------------- System "+ system+ " training WITHOUT features importance ")
    print("\n--------------- Start time:: " + str(pd.Timestamp.now()))
    print("\n-----------------------------------------------------------------------------")
    df_result_nfi = run_Algorithms(None, False, system)
    print("\n\n-----------------------------------------------------------------------------")
    print("\n--------------- System "+ system+ " training ")
    print("\n--------------- End time:: " + str(pd.Timestamp.now()))
    print("\n-----------------------------------------------------------------------------")
    display(df_result_nfi)
    
    print("\n\n-----------------------------------------------------------------------------")
    print("\n--------------- System "+ system+ " training WITH features importance ")
    print("\n--------------- Start time:: " + str(pd.Timestamp.now()))
    print("\n-----------------------------------------------------------------------------")
    df_result_fi = run_Algorithms(None, True, system)
    print("\n\n-----------------------------------------------------------------------------")
    print("\n--------------- System "+ system+ " training ")
    print("\n--------------- End time:: " + str(pd.Timestamp.now()))
    print("\n-----------------------------------------------------------------------------")
    display(df_result_fi)

    

LLVM


-----------------------------------------------------------------------------

--------------- System LLVM training WITHOUT features importance 

--------------- Start time:: 2023-06-15 13:20:08.789982

-----------------------------------------------------------------------------
Deleted columns: 


['root', 'time_passes']

Sample features number: 4


count    18.000000
mean      2.884592
std       3.371602
min       0.049350
25%       0.721292
50%       1.652877
75%       3.469307
max      13.173072
Name: % error, dtype: float64

count    18.000000
mean      2.953826
std       3.426079
min       0.104700
25%       0.714095
50%       1.727119
75%       3.717596
max      13.304340
Name: % error, dtype: float64

...

count    149.000000
mean       2.857991
std        3.353833
min        0.022142
25%        0.935513
50%        1.875578
75%        3.623484
max       20.209675
Name: % error, dtype: float64

['twise',
 0.6,
 7,
 True,
 'RandomForest',
 'RandomForest',
 None,
 None,
 Timestamp('2023-06-15 15:37:54.823082'),
 Timestamp('2023-06-15 15:41:43.668178')]

,algorithm_name,ft_dummyfication,origin_ft_selection,ft_selection,hyperparameters,size_target,test_size,coef_order,accuracy
0,LinearRegression,False,None,"Index(['gvn', 'instcombine', 'inline', 'jump_t...",LinearRegression(),Performance,0.1,licm 10.511864 gvn ...,count 18.000000 mean 2.884592 std ...
1,Lasso,False,None,"Index(['gvn', 'instcombine', 'inline', 'jump_t...","Lasso(alpha=0.1, max_iter=100)",Performance,0.1,licm 13.439089 gvn ...,count 18.000000 mean 2.953826 std ...
2,Ridge,False,None,"Index(['gvn', 'instcombine', 'inline', 'jump_t...",Ridge(),Performance,0.1,licm 10.199090 gvn ...,count 18.000000 mean 2.937313 std ...
3,ElasticNet,False,None,"Index(['gvn', 'instcombine', 'inline', 'jump_t...","ElasticNet(alpha=0.1, l1_ratio=1.0, max_iter=100)",Performance,0.1,licm 13.439089 gvn ...,count 18.000000 mean 2.953826 std ...
4,PolynomialRegression,False,None,"Index(['gvn', 'instcombine', 'inline', 'jump_t...",LinearRegression(),Performance,0.1,licm 13.439089 gvn ...,count 18.000000 mean 2.953826 std ...
...,...,...,...,...,...,...,...,...,...
5179,Lasso,False,RandomForest,"Index(['gvn', 'instcombine', 'inline', 'licm',...","Lasso(alpha=0.3, max_iter=5)",Performance,0.9,gvn 26.222435 instcombine ...,count 149.000000 mean 2.209623 std ...
5180,Ridge,False,RandomForest,"Index(['gvn', 'instcombine', 'inline', 'licm',...",Ridge(alpha=0.5),Performance,0.9,gvn 22.418108 instcombine ...,count 149.000000 mean 2.325296 std ...
5181,ElasticNet,False,RandomForest,"Index(['gvn', 'instcombine', 'inline', 'licm',...","ElasticNet(alpha=0.3, l1_ratio=1.0, max_iter=5)",Performance,0.9,gvn 26.222435 instcombine ...,count 149.000000 mean 2.209623 std ...
5182,PolynomialRegression,False,RandomForest,"Index(['gvn', 'instcombine', 'inline', 'licm',...",LinearRegression(),Performance,0.9,gvn 23.775782 inline ...,count 149.000000 mean 2.866599 std ...




-----------------------------------------------------------------------------

--------------- System LLVM training 

--------------- End time:: 2023-06-15 15:42:17.387069

-----------------------------------------------------------------------------


,algorithm_name,ft_dummyfication,origin_ft_selection,ft_selection,hyperparameters,size_target,test_size,coef_order,accuracy
0,LinearRegression,False,None,"Index(['gvn', 'instcombine', 'inline', 'jump_t...",LinearRegression(),Performance,0.1,licm 10.511864 gvn ...,count 18.000000 mean 2.884592 std ...
1,Lasso,False,None,"Index(['gvn', 'instcombine', 'inline', 'jump_t...","Lasso(alpha=0.1, max_iter=100)",Performance,0.1,licm 13.439089 gvn ...,count 18.000000 mean 2.953826 std ...
2,Ridge,False,None,"Index(['gvn', 'instcombine', 'inline', 'jump_t...",Ridge(),Performance,0.1,licm 10.199090 gvn ...,count 18.000000 mean 2.937313 std ...
3,ElasticNet,False,None,"Index(['gvn', 'instcombine', 'inline', 'jump_t...","ElasticNet(alpha=0.1, l1_ratio=1.0, max_iter=100)",Performance,0.1,licm 13.439089 gvn ...,count 18.000000 mean 2.953826 std ...
4,PolynomialRegression,False,None,"Index(['gvn', 'instcombine', 'inline', 'jump_t...",LinearRegression(),Performance,0.1,licm 13.439089 gvn ...,count 18.000000 mean 2.953826 std ...
...,...,...,...,...,...,...,...,...,...
5179,Lasso,False,RandomForest,"Index(['gvn', 'instcombine', 'inline', 'licm',...","Lasso(alpha=0.3, max_iter=5)",Performance,0.9,gvn 26.222435 instcombine ...,count 149.000000 mean 2.209623 std ...
5180,Ridge,False,RandomForest,"Index(['gvn', 'instcombine', 'inline', 'licm',...",Ridge(alpha=0.5),Performance,0.9,gvn 22.418108 instcombine ...,count 149.000000 mean 2.325296 std ...
5181,ElasticNet,False,RandomForest,"Index(['gvn', 'instcombine', 'inline', 'licm',...","ElasticNet(alpha=0.3, l1_ratio=1.0, max_iter=5)",Performance,0.9,gvn 26.222435 instcombine ...,count 149.000000 mean 2.209623 std ...
5182,PolynomialRegression,False,RandomForest,"Index(['gvn', 'instcombine', 'inline', 'licm',...",LinearRegression(),Performance,0.9,gvn 23.775782 inline ...,count 149.000000 mean 2.866599 std ...


In [ ]:
df_result['accuracy'][0]

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

rs = []
for i in range(len(df_result_fi['accuracy'])):
    rs.append(df_result_fi['accuracy'][i]['mean'])

# rs
# display (df_result['algorithm_name'], df_result['accuracy'], rs)

plt.scatter(df_result_fi['algorithm_name'], rs, color ="blue", marker = "s")
plt.plot(df_result_fi['algorithm_name'], rs, color ="blue")


In [ ]:
# Boxplot
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


data = [model['LinearRegression'].coef_, model['Lasso'].coef_, model['Ridge'].coef_, model['ElasticNet'].coef_, model['PolynomialRegression'].coef_]
fig1, ax1 = plt.subplots()
ax1.set_title('Coeficient of feature importance for all algorithms')
ax1.boxplot(data)
plt.show()